In [32]:
import pandas as pd


clustering_algorithms = ['gaussian_mixture', 'kmeans']
fusion_types = ["add", "cat"]
weights = ["0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9"]
iteration = 30
class_num = ["23", "34"]
conv_types = ["se", "re"]
eval_methods = ['ari', 'nmi', 'purity']

name_column = []
for c_num in class_num:
    data_name = "rico_" + c_num
    name_column.append(data_name)
    row_name = 'seq2seq_' + c_num
    name_column.append(row_name)
    for c_type in conv_types:
        row_name = 'conv_' + c_type + '_' + c_num
        name_column.append(row_name)
    for f_type in fusion_types:
        row_name = 'rico' + c_num + '_seq2seq' + c_num + '_' + f_type
        name_column.append(row_name)
    for f_type in fusion_types:
        row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type
        name_column.append(row_name)
    for c_type in conv_types:
        for f_type in fusion_types:
            row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
            name_column.append(row_name)
        for f_type in fusion_types:
            row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
            name_column.append(row_name)
    for f_type in fusion_types:
        for w in weights:
            row_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + f_type + "_" + w
            name_column.append(row_name)
    for f_type in fusion_types:
        for w in weights:
            row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type + "_" + w
            name_column.append(row_name)
    for c_type in conv_types:
        for w in weights:
            for f_type in fusion_types:
                row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w             
                name_column.append(row_name)
            for f_type in fusion_types:
                row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w
                name_column.append(row_name)

column_name = [str(x) for x in range(30)]
df_ari_gmm = pd.DataFrame(columns=column_name)
df_ari_gmm = df_ari_gmm.assign(name = name_column)
df_ari_gmm.set_index('name', inplace=True)
df_ari_km = pd.DataFrame(columns=column_name)
df_ari_km = df_ari_km.assign(name = name_column)
df_ari_km.set_index('name', inplace=True)

df_nmi_gmm = pd.DataFrame(columns=column_name)
df_nmi_gmm = df_ari_gmm.assign(name = name_column)
df_nmi_gmm.set_index('name', inplace=True)
df_nmi_km = pd.DataFrame(columns=column_name)
df_nmi_km = df_ari_km.assign(name = name_column)
df_nmi_km.set_index('name', inplace=True)

df_purity_gmm = pd.DataFrame(columns=column_name)
df_purity_gmm = df_ari_gmm.assign(name = name_column)
df_purity_gmm.set_index('name', inplace=True)
df_purity_km = pd.DataFrame(columns=column_name)
df_purity_km = df_ari_km.assign(name = name_column)
df_purity_km.set_index('name', inplace=True)

def run(data_name, iter, row_name):    
    lres = []
    for i, eval_method in enumerate(eval_methods):
        csv_path = 'csv/' + eval_method + '/' + data_name + ".csv"
        with open(csv_path, 'r') as fd:
            res = fd.read()
        lres = res.split('\n')
        if eval_method == 'purity':
            temp = lres[-2].split('Total,')[-1].split(',')
            df_purity_gmm.loc[row_name, str(iter)] = float(temp[0].strip('\%'))/100
            df_purity_km.loc[row_name, str(iter)] = float(temp[1].strip('\%'))/100
        elif eval_method == 'nmi':
            temp = lres[-2].split(',')
            df_nmi_gmm.loc[row_name, str(iter)] = float(temp[0])
            df_nmi_km.loc[row_name, str(iter)] = float(temp[1])
        else:
            temp = lres[-2].split(',')
            df_ari_gmm.loc[row_name, str(iter)] = float(temp[0])
            df_ari_km.loc[row_name, str(iter)] = float(temp[1])

for c_num in class_num:
    data_name = "rico_" + c_num
    run(data_name, 0, data_name)
    for _iter in range(iteration):
        data_name = "seq2seq_" + c_num + "_" + str(_iter)
        row_name = 'seq2seq_' + c_num
        run(data_name, _iter, row_name)
        for c_type in conv_types:
            data_name = "conv_" + c_type + "_" + c_num + "_" + str(_iter)
            row_name = 'conv_' + c_type + '_' + c_num
            run(data_name, _iter, row_name)
        for f_type in fusion_types:
            data_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + str(_iter) + "_" + f_type
            row_name = 'rico' + c_num + '_seq2seq' + c_num + '_' + f_type
            run(data_name, _iter, row_name)
        for f_type in fusion_types:
            data_name = "conv_re_" + c_num + "_" + str(_iter) + "_conv_se_" + c_num + "_" + str(_iter) + "_" + f_type
            row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type
            run(data_name, _iter, row_name)
        for c_type in conv_types:
            for f_type in fusion_types:
                data_name = ("rico_" + c_num + "_conv_" + c_type + "_" +
                                c_num + "_" + str(_iter) + "_" + f_type)
                row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
                run(data_name, _iter, row_name)
            for f_type in fusion_types:
                data_name = ("seq2seq_" + c_num + "_" + str(_iter) + "_conv_" + c_type + "_" +
                                c_num + "_" + str(_iter) + "_" + f_type)
                row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
                run(data_name, _iter, row_name)
        for f_type in fusion_types:
            for w in weights:
                data_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + str(_iter) + "_" + f_type + "_" + w
                row_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + f_type + "_" + w
                run(data_name, _iter, row_name)
        for f_type in fusion_types:
            for w in weights:
                data_name = "conv_re_" + c_num + "_" + str(_iter) + "_conv_se_" + c_num + "_" + str(_iter) + "_" + f_type + "_" + w
                row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type + "_" + w
                run(data_name, _iter, row_name)
        for c_type in conv_types:
            for w in weights:
                for f_type in fusion_types:
                    data_name = ("rico_" + c_num + "_conv_" + c_type + "_" +
                                    c_num + "_" + str(_iter) + "_" + f_type + "_" + w)
                    row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w             
                    run(data_name, _iter, row_name)
                for f_type in fusion_types:
                    data_name = ("seq2seq_" + c_num + "_" + str(_iter) + "_conv_" + c_type + "_" +
                                    c_num + "_" + str(_iter) + "_" + f_type + "_" + w)
                    row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w
                    run(data_name, _iter, row_name)


In [33]:
df_ari_gmm['Mean'] = df_ari_gmm.mean(axis=1)
df_ari_gmm['Std'] = df_ari_gmm.std(axis=1)
df_ari_km['Mean'] = df_ari_km.mean(axis=1)
df_ari_km['Std'] = df_ari_km.std(axis=1)
df_nmi_gmm['Mean'] = df_nmi_gmm.mean(axis=1)
df_nmi_gmm['Std'] = df_nmi_gmm.std(axis=1)
df_nmi_km['Mean'] = df_nmi_km.mean(axis=1)
df_nmi_km['Std'] = df_nmi_km.std(axis=1)
df_purity_gmm['Mean'] = df_purity_gmm.mean(axis=1)
df_purity_gmm['Std'] = df_purity_gmm.std(axis=1)
df_purity_km['Mean'] = df_purity_km.mean(axis=1)
df_purity_km['Std'] = df_purity_km.std(axis=1)

In [34]:
df_purity_km

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,Mean,Std
name,,,,,,,,,,,,,,,,,,,,,
rico_23,0.326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.326000,0.000000
seq2seq_23,0.313,0.386,0.403,0.387,0.336,0.379,0.396,0.387,0.387,0.363,...,0.373,0.356,0.27,0.341,0.361,0.369,0.323,0.394,0.369467,0.031674
conv_se_23,0.396,0.369,0.424,0.333,0.473,0.456,0.433,0.46,0.42,0.459,...,0.419,0.476,0.394,0.334,0.433,0.373,0.353,0.407,0.411133,0.040780
conv_re_23,0.249,0.251,0.246,0.261,0.27,0.287,0.244,0.227,0.229,0.263,...,0.23,0.227,0.23,0.306,0.22,0.279,0.291,0.243,0.253033,0.029757
rico23_seq2seq23_add,0.371,0.424,0.464,0.376,0.459,0.454,0.416,0.389,0.423,0.409,...,0.389,0.419,0.36,0.42,0.463,0.336,0.389,0.389,0.406600,0.034856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
seq2seq_34_conv_re_34_cat_0.8,0.344,0.321,0.366,0.329,0.321,0.329,0.299,0.301,0.278,0.329,...,0.253,0.326,0.295,0.385,0.344,0.324,0.34,0.328,0.325533,0.027777
rico_34_conv_re_34_add_0.9,0.335,0.257,0.358,0.343,0.317,0.298,0.267,0.31,0.315,0.375,...,0.266,0.296,0.284,0.322,0.333,0.279,0.33,0.307,0.316633,0.033870
rico_34_conv_re_34_cat_0.9,0.343,0.284,0.299,0.322,0.298,0.329,0.299,0.319,0.277,0.383,...,0.253,0.306,0.269,0.316,0.368,0.329,0.356,0.31,0.322333,0.032766


In [35]:
df_ari_gmm.to_csv('ari_gmm.csv')
df_ari_km.to_csv('ari_km.csv')
df_nmi_gmm.to_csv('nmi_gmm.csv')
df_nmi_km.to_csv('nmi_km.csv')
df_purity_gmm.to_csv('purity_gmm.csv')
df_purity_km.to_csv('purity_km.csv')
